In [1]:
import numpy as np

from skimage.io import imshow
from skimage.transform import resize

from sklearn.model_selection import train_test_split

from collections import namedtuple

import threading

from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Activation
from keras.optimizers import Adam
from keras.optimizers import SGD

from augmentation import ImageGenerator
%matplotlib inline

Using TensorFlow backend.


In [2]:
def get_train_data(path='public_data/00_input/train/'):
    from skimage.io import imread
    with open(path + 'gt.csv') as navigation_file:
        splits = [line.split(',') for line in navigation_file]
        points_number = int((len(splits[0]) - 1) / 2)
        train_x = np.array([imread(path + 'images/' + split[0]) for split in splits[1:]])
        train_y = np.array([[int(x) for x in split[1:]] for split in splits[1:]])
        return train_x, train_y, points_number

def get_colored_data(images, coordinates):
    return (
        np.array([image for image in images if len(np.shape(image)) == 3]),
        np.array([answer for image, answer in zip(images, coordinates) if len(np.shape(image)) == 3])
    )

def get_resized_train_data(train_x, train_y, shape=(100, 100, 3)):
    resized_x = np.array([resize(image, shape, mode='reflect') for image in train_x])
    resized_y = np.array([[
            int(coordinate * shape[i % 2] / np.shape(image)[i % 2])
            for i, coordinate in enumerate(answer)
        ] for image, answer in zip(train_x, train_y)
    ])
    return np.array(resized_x), np.array(resized_y)

In [3]:
all_x, all_y, points_number = get_train_data()
colored_x, colored_y = get_colored_data(all_x, all_y)
resized_x, resized_y = get_resized_train_data(colored_x, colored_y)
input_shape=np.shape(resized_x[0])
train_x, test_x, train_y, test_y = train_test_split(
    resized_x, resized_y, test_size=0.3, random_state=42
)

datagen = ImageGenerator()

In [4]:
datagen = ImageDataGenerator()

In [5]:
regularization_lambda = 5e-6

def add_Conv2D_relu(model, n_filter, filters_size, input_shape=None):
    if input_shape is not None:
        model.add(Conv2D(
            n_filter, filters_size, padding='same',
            kernel_regularizer=regularizers.l2(regularization_lambda),
            activation='elu', input_shape=input_shape,
        ))
    else:
        model.add(Conv2D(
            n_filter, filters_size, padding='same',
            kernel_regularizer=regularizers.l2(regularization_lambda),
            activation='elu'
        ))

model = Sequential()

add_Conv2D_relu(model, 64, (3, 3), input_shape)
add_Conv2D_relu(model, 64, (3, 3))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

add_Conv2D_relu(model, 128, (3, 3))
add_Conv2D_relu(model, 128, (3, 3))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

add_Conv2D_relu(model, 256, (3, 3))
add_Conv2D_relu(model, 256, (3, 3))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

model.add(Flatten(name='flatten'))
model.add(Dense(512, activation='elu', kernel_regularizer=regularizers.l2(regularization_lambda)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='elu', kernel_regularizer=regularizers.l2(regularization_lambda)))

model.add(Dense(points_number * 2))

In [6]:
model.compile(loss='mean_squared_error',
              optimizer=Adam(),
              metrics=['mse'])

In [7]:
class MyClallback:
    def __init__(self, model, directory):
        self._model = model
        self._directory = directory
        self._counter = 0

    def set_model(self, model):
        pass

    def set_params(self, params):
        pass

    def on_train_begin(self, logs):
        pass

    def on_train_end(self, logs):
        pass

    def on_epoch_begin(self, epoch, logs):
        pass

    def on_epoch_end(self, epoch, logs):
        self._model.save(
            self._directory + '/' + str(self._counter) +
            '_mse:_' + str(logs['mean_squared_error']) +
            '_val_mse:_' + str(logs['val_mean_squared_error']) +
            '.npz'
        )
        self._counter += 1

    def on_batch_begin(self, batch, logs):
        pass

    def on_batch_end(self, batch, logs):
        pass

In [8]:
batch_size = 32
model.fit_generator(datagen.flow(train_x, train_y, batch_size=batch_size),
                    steps_per_epoch=int(len(train_y) / batch_size), epochs=1000,
                    callbacks=[MyClallback(model, 'no_aug_models')], validation_data=(test_x, test_y))

Epoch 1/1000
130/130 [==============================] - 49s - loss: 153.0427 - mean_squared_error: 153.0285 - val_loss: 526.1290 - val_mean_squared_error: 526.1140
Epoch 2/1000
130/130 [==============================] - 33s - loss: 40.5867 - mean_squared_error: 40.5714 - val_loss: 64.3035 - val_mean_squared_error: 64.2880
Epoch 3/1000
130/130 [==============================] - 33s - loss: 24.8557 - mean_squared_error: 24.8399 - val_loss: 21.8698 - val_mean_squared_error: 21.8537
Epoch 4/1000
130/130 [==============================] - 33s - loss: 21.1852 - mean_squared_error: 21.1688 - val_loss: 19.0066 - val_mean_squared_error: 18.9900
Epoch 5/1000
130/130 [==============================] - 34s - loss: 19.6063 - mean_squared_error: 19.5893 - val_loss: 16.8888 - val_mean_squared_error: 16.8715
Epoch 6/1000
130/130 [==============================] - 34s - loss: 16.7936 - mean_squared_error: 16.7760 - val_loss: 14.7952 - val_mean_squared_error: 14.7774
Epoch 7/1000
130/130 [==============

130/130 [==============================] - 35s - loss: 23.2907 - mean_squared_error: 23.1831 - val_loss: 24.0883 - val_mean_squared_error: 23.9806
Epoch 52/1000
130/130 [==============================] - 35s - loss: 22.8994 - mean_squared_error: 22.7916 - val_loss: 23.9735 - val_mean_squared_error: 23.8656
Epoch 53/1000
130/130 [==============================] - 35s - loss: 23.6430 - mean_squared_error: 23.5350 - val_loss: 23.1430 - val_mean_squared_error: 23.0349
Epoch 54/1000
130/130 [==============================] - 35s - loss: 22.3944 - mean_squared_error: 22.2863 - val_loss: 23.0337 - val_mean_squared_error: 22.9255
Epoch 55/1000
130/130 [==============================] - 35s - loss: 22.9997 - mean_squared_error: 22.8913 - val_loss: 23.2428 - val_mean_squared_error: 23.1344
Epoch 56/1000
130/130 [==============================] - 35s - loss: 22.5335 - mean_squared_error: 22.4250 - val_loss: 22.8045 - val_mean_squared_error: 22.6959
Epoch 57/1000
130/130 [=========================

130/130 [==============================] - 35s - loss: 12.8366 - mean_squared_error: 12.7147 - val_loss: 18.7199 - val_mean_squared_error: 18.5981
Epoch 102/1000
130/130 [==============================] - 35s - loss: 12.3524 - mean_squared_error: 12.2306 - val_loss: 17.7674 - val_mean_squared_error: 17.6454
Epoch 103/1000
130/130 [==============================] - 35s - loss: 12.8825 - mean_squared_error: 12.7604 - val_loss: 16.6895 - val_mean_squared_error: 16.5673
Epoch 104/1000
130/130 [==============================] - 35s - loss: 12.6730 - mean_squared_error: 12.5508 - val_loss: 16.7826 - val_mean_squared_error: 16.6604
Epoch 105/1000
130/130 [==============================] - 35s - loss: 12.1892 - mean_squared_error: 12.0668 - val_loss: 16.8654 - val_mean_squared_error: 16.7427
Epoch 106/1000
130/130 [==============================] - 35s - loss: 12.9130 - mean_squared_error: 12.7899 - val_loss: 17.4838 - val_mean_squared_error: 17.3603
Epoch 107/1000
130/130 [===================

130/130 [==============================] - 35s - loss: 9.8811 - mean_squared_error: 9.7374 - val_loss: 14.5229 - val_mean_squared_error: 14.3795
Epoch 152/1000
130/130 [==============================] - 35s - loss: 10.2044 - mean_squared_error: 10.0605 - val_loss: 14.0223 - val_mean_squared_error: 13.8780
Epoch 153/1000
130/130 [==============================] - 35s - loss: 11.1354 - mean_squared_error: 10.9911 - val_loss: 13.6051 - val_mean_squared_error: 13.4605
Epoch 154/1000
130/130 [==============================] - 35s - loss: 10.2138 - mean_squared_error: 10.0690 - val_loss: 13.3221 - val_mean_squared_error: 13.1775
Epoch 155/1000
130/130 [==============================] - 35s - loss: 9.7204 - mean_squared_error: 9.5760 - val_loss: 13.7865 - val_mean_squared_error: 13.6424
Epoch 156/1000
130/130 [==============================] - 35s - loss: 10.3305 - mean_squared_error: 10.1863 - val_loss: 38.7277 - val_mean_squared_error: 38.5823
Epoch 157/1000
130/130 [=======================

130/130 [==============================] - 35s - loss: 9.2273 - mean_squared_error: 9.0327 - val_loss: 12.7091 - val_mean_squared_error: 12.5143
Epoch 203/1000
130/130 [==============================] - 35s - loss: 8.8670 - mean_squared_error: 8.6721 - val_loss: 12.3543 - val_mean_squared_error: 12.1597
Epoch 204/1000
130/130 [==============================] - 35s - loss: 8.3982 - mean_squared_error: 8.2037 - val_loss: 12.3730 - val_mean_squared_error: 12.1786
Epoch 205/1000
130/130 [==============================] - 35s - loss: 9.7130 - mean_squared_error: 9.5177 - val_loss: 12.5479 - val_mean_squared_error: 12.3520
Epoch 206/1000
130/130 [==============================] - 35s - loss: 9.0250 - mean_squared_error: 8.8292 - val_loss: 12.4521 - val_mean_squared_error: 12.2562
Epoch 207/1000
130/130 [==============================] - 35s - loss: 8.1524 - mean_squared_error: 7.9567 - val_loss: 12.3276 - val_mean_squared_error: 12.1320
Epoch 208/1000
130/130 [==============================]

130/130 [==============================] - 35s - loss: 7.6544 - mean_squared_error: 7.4578 - val_loss: 11.2769 - val_mean_squared_error: 11.0806
Epoch 254/1000
130/130 [==============================] - 35s - loss: 7.5928 - mean_squared_error: 7.3965 - val_loss: 11.3596 - val_mean_squared_error: 11.1634
Epoch 255/1000
130/130 [==============================] - 35s - loss: 7.7332 - mean_squared_error: 7.5368 - val_loss: 11.4670 - val_mean_squared_error: 11.2706
Epoch 256/1000
130/130 [==============================] - 35s - loss: 7.8095 - mean_squared_error: 7.6134 - val_loss: 12.4322 - val_mean_squared_error: 12.2363
Epoch 257/1000
130/130 [==============================] - 35s - loss: 7.4781 - mean_squared_error: 7.2823 - val_loss: 11.1274 - val_mean_squared_error: 10.9319
Epoch 258/1000
130/130 [==============================] - 35s - loss: 7.5054 - mean_squared_error: 7.3105 - val_loss: 11.1431 - val_mean_squared_error: 10.9486
Epoch 259/1000
130/130 [==============================]

130/130 [==============================] - 35s - loss: 6.6566 - mean_squared_error: 6.4663 - val_loss: 10.8821 - val_mean_squared_error: 10.6912
Epoch 305/1000
130/130 [==============================] - 36s - loss: 7.4464 - mean_squared_error: 7.2556 - val_loss: 11.2016 - val_mean_squared_error: 11.0108
Epoch 306/1000
130/130 [==============================] - 36s - loss: 8.0445 - mean_squared_error: 7.8462 - val_loss: 12.1175 - val_mean_squared_error: 11.9161
Epoch 307/1000
130/130 [==============================] - 36s - loss: 6.8323 - mean_squared_error: 6.6303 - val_loss: 11.7576 - val_mean_squared_error: 11.5553
Epoch 308/1000
130/130 [==============================] - 36s - loss: 6.5345 - mean_squared_error: 6.3318 - val_loss: 11.7024 - val_mean_squared_error: 11.4997
Epoch 309/1000
130/130 [==============================] - 36s - loss: 6.6185 - mean_squared_error: 6.4155 - val_loss: 11.2778 - val_mean_squared_error: 11.0745
Epoch 310/1000
130/130 [==============================]

KeyboardInterrupt: 